Notes

Now Using Python / Jupyter

Plan on splitting work into four parts
1. Data Scrapping
2. Data Cleaning / Filtering
3. Data Visualizations
4. Analytics

Libraries
- pandas: data manipulation and analysis
- numpy, matplotlib, seaborn: statistical data visualization
- beautifulsoup: python library for working with HTML and XML

Changes:
- Instead of using the downloaded playerset from kaggle, I decided it would be more of a challenge and more interesting to pull straight from pro-football-reference.com (which the kaggle pulled from anyway)
- This will also allow me to pull the data I need exactly instead of sifting through 80+ years of data which takes up space and takes time. 


In [1]:
import numpy as np
import pandas as pd

#shell interface similar to matlab
import matplotlib.pyplot as plt

#data viz lib similar to matlab
import seaborn

#html and xml lib
from bs4 import BeautifulSoup

#http lib 
import requests

1. Data Scrapping

In [2]:
url = ["https://www.pro-football-reference.com/years/2007/passing.htm", 
       "https://www.pro-football-reference.com/years/2011/passing.htm"]
yr = 2007
dfs = []

for i in range(0,2):
    #retrieves the page
    re = requests.get(url[i])
    root = BeautifulSoup(re.content)

    #finds the table
    t = root.find("table")

    #convert table to data frame
    df = pd.read_html(str(t))[0]

    #add the year for each player
    df["year"] = yr
    dfs.append(df)
    
    i+1
    yr = 2011

dfs

[      Rk          Player   Tm Age Pos   G  GS   QBrec  Cmp  Att  ...   Rate  \
 0      1     Tom Brady*+  NWE  30  QB  16  16  16-0-0  398  578  ...  117.2   
 1      2      Drew Brees  NOR  28  QB  16  16   7-9-0  440  652  ...   89.4   
 2      3      Tony Romo*  DAL  27  QB  16  16  13-3-0  335  520  ...   97.4   
 3      4    Brett Favre*  GNB  38  QB  16  16  13-3-0  356  535  ...   95.7   
 4      5   Carson Palmer  CIN  28  QB  16  16   7-9-0  373  575  ...   86.7   
 ..   ...             ...  ...  ..  ..  ..  ..     ...  ...  ...  ...    ...   
 99    97      Greg Lewis  PHI  27  wr  15   1     NaN    0    1  ...   39.6   
 100   98   Mewelde Moore  MIN  25  rb  12   0     NaN    0    1  ...   39.6   
 101   99  Willie Parker*  PIT  27  RB  15  15     NaN    0    1  ...   39.6   
 102  100      Brad Smith  NYJ  24  WR  16   9     NaN    0    1  ...   39.6   
 103  101  Chester Taylor  MIN  28  rb  14   8     NaN    0    1  ...   39.6   
 
       QBR  Sk Yds.1  Sk%  NY/A ANY/A 

2. Cleaning

In [3]:
#new array filtering out only qbs
qbs = []

#remove repeating column headers
for i in range(0, len(dfs)):
    df = dfs[i]
    df = df[~df["Rk"].str.contains("Rk")]
    
    #filter out qbs based on QBrec and games started
    df["QBrec"] = df["QBrec"].astype(str) #convert to string
    df = df[~df["QBrec"].str.contains("NaN")]
    df = df[~df["QBrec"].str.contains("nan")]
    df["GS"] = pd.to_numeric(df["GS"]) #convert to number
    df = df[df["GS"] > 7] 
    
    qbs.append(df)

qbs

<ipython-input-3-3b1ef1cb88c9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["QBrec"] = df["QBrec"].astype(str) #convert to string


[    Rk               Player   Tm Age Pos   G  GS   QBrec  Cmp  Att  ...  \
 0    1          Tom Brady*+  NWE  30  QB  16  16  16-0-0  398  578  ...   
 1    2           Drew Brees  NOR  28  QB  16  16   7-9-0  440  652  ...   
 2    3           Tony Romo*  DAL  27  QB  16  16  13-3-0  335  520  ...   
 3    4         Brett Favre*  GNB  38  QB  16  16  13-3-0  356  535  ...   
 4    5        Carson Palmer  CIN  28  QB  16  16   7-9-0  373  575  ...   
 5    6            Jon Kitna  DET  35  QB  16  16   7-9-0  355  561  ...   
 6    7      Peyton Manning*  IND  31  QB  16  16  13-3-0  337  515  ...   
 7    8     Matt Hasselbeck*  SEA  32  QB  16  16  10-6-0  352  562  ...   
 8    9      Derek Anderson*  CLE  24  QB  16  15  10-5-0  298  527  ...   
 9   10           Jay Cutler  DEN  24  QB  16  16   7-9-0  297  467  ...   
 10  11          Kurt Warner  ARI  36  QB  14  11   5-6-0  281  451  ...   
 11  12          Eli Manning  NYG  26  QB  16  16  10-6-0  297  529  ...   
 12  13     

In [4]:
#combine 2007 and 2011 tables into one
QB = qbs[0]
for i in range (1, len(qbs)):
    QB = QB.append(qbs[i])

QB

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,year
0,1,Tom Brady*+,NWE,30,QB,16,16,16-0-0,398,578,...,117.2,87.0,21,128,3.5,7.81,8.88,4,4,2007
1,2,Drew Brees,NOR,28,QB,16,16,7-9-0,440,652,...,89.4,64.6,16,109,2.4,6.46,6.08,1,1,2007
2,3,Tony Romo*,DAL,27,QB,16,16,13-3-0,335,520,...,97.4,76.9,24,176,4.4,7.42,7.17,2,2,2007
3,4,Brett Favre*,GNB,38,QB,16,16,13-3-0,356,535,...,95.7,70.8,15,93,2.7,7.39,7.18,2,4,2007
4,5,Carson Palmer,CIN,28,QB,16,16,7-9-0,373,575,...,86.7,62.5,17,119,2.9,6.78,6.14,2,2,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,30,Christian Ponder,MIN,23,QB,11,10,2-8-0,158,291,...,70.1,32.4,30,164,9.3,5.26,4.25,0,1,2011
31,31,Kyle Orton,2TM,29,NaN,9,8,3-5-0,150,252,...,77.8,NaN,10,54,3.8,6.50,5.65,NaN,NaN,2011
32,32,Tim Tebow,DEN,24,QB,14,11,7-4-0,126,271,...,72.9,38.6,33,225,10.9,4.95,4.85,5,5,2011
33,33,Matt Cassel,KAN,29,QB,9,9,4-5-0,160,269,...,76.6,48.4,22,120,7.6,5.47,4.77,1,2,2011


In [ ]:
# convert stats to a per game instead of season total
# filter out irrelevant / not useful (Rk, Tm, Age, Pos, etc)

